# Libraries and Utilities

In [2]:
import re
import nltk
import pickle
import difflib
import warnings
import numpy as np
import pandas as pd

# to ignore all the warnings
warnings.filterwarnings('ignore')

from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Loading and Checking the data

In [3]:
books = pd.read_csv('dataset/Preprocessed_data.csv')
books.head(3)

,Unnamed: 0,user_id,location,age,isbn,rating,book_title,book_author,year_of_publication,publisher,img_s,img_m,img_l,Summary,Language,Category,city,state,country
0,0,2,"stockton, california, usa",18.0000,0195153448,0,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,Provides an introduction to classical myths pl...,en,['Social Science'],stockton,california,usa
1,1,8,"timmins, ontario, canada",34.7439,0002005018,5,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],timmins,ontario,canada
2,2,11400,"ottawa, ontario, canada",49.0000,0002005018,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],ottawa,ontario,canada


# Data Preprocessing

In [4]:
df = books.copy()

# dropping null values and useless columns
df.dropna(inplace = True)
df.drop(columns = ['Unnamed: 0', 'user_id', 'location', 'age', 'isbn', 'year_of_publication', 'img_m',
                  'img_s', 'Language', 'city', 'state', 'country'], inplace = True)

# dropping rows with rating 0 and category 9
df.drop(index=df[df['rating'] == 0].index, inplace=True)
df.drop(index=df[df['Category'] == '9'].index, inplace=True)

# formatting the category column to keep only strings
df['Category'] = df['Category'].apply(lambda x: re.sub('[\W_]+',' ',x).strip())
df.reset_index(drop = True, inplace = True)

# cutting the dataset small by removing books having less than 10 reviews
rating_counts = pd.DataFrame(df['book_title'].value_counts())
rare_books = rating_counts[rating_counts['book_title'] <= 15].index
common_books = df[~df['book_title'].isin(rare_books)]

# droping duplicates so that one book is in one row
book_list = common_books.drop_duplicates(subset = ['book_title'])

#since now we do not need ratings we can remove rating column as well
book_list.drop(columns = ['rating'], inplace = True)
book_list.reset_index(inplace = True)
book_list.head()
print(book_list.shape)

(1854, 7)


# Finding similarity of the books with each other

In [5]:
# filtering the summary for common english words and removing them
filtered_summary = []
for i in book_list['Summary']:
               
    i = re.sub("[^a-zA-Z]"," ",i).lower()
    i = nltk.word_tokenize(i)
    i = [word for word in i if not word.lower() in set(stopwords.words("english"))]
    i = " ".join(i)
    filtered_summary.append(i)
        
book_list['Summary'] = filtered_summary

# combining all the features to use it as a single feature
book_list['combined_features'] = ''
combined_features = ['book_title', 'book_author', 'publisher', 'Category', 'Summary']
 
for feature in combined_features : 
    book_list['combined_features'] += (book_list[feature] + ' ')
         
book_list.drop(columns = ['index'], inplace = True)
book_list.reset_index(inplace = True)
  
# finding feature vectors using vectorizer
book_list['feature_vectors'] = ''
tf_vectorizer = TfidfVectorizer()
book_feature_vectors = tf_vectorizer.fit_transform(book_list['combined_features'])
   
# finding the cosine similarity for each book with every other book.
# similarity_matrix is a square matrix with shape [(book_list.length() x book_list.length())]
cosine_similarity_matrix = cosine_similarity(book_feature_vectors)

# Pickle and exporting the model

In [7]:
pickle.dump(cosine_similarity_matrix, open('similarity_matrix.pkl', 'wb'))
pickle.dump(book_list, open('book_list.pkl', 'wb'))
pickle.dump(user_data, open('user_data.pkl', 'wb'))